In [1]:
# Imports
import warnings
warnings.filterwarnings("ignore")
# Import libraries
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil import relativedelta
import operator
import os
import random
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
import matplotlib.ticker as ticker
from dateutil import relativedelta
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: f'%.{len(str(x%1))-2}f' % x)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [2]:
import numpy as np

def otg_aggregates(CIF_ID, APPLICABLE_DATE, df):
    aggr_df = df[(df['CIF_ID'] == CIF_ID) & (df['TRAN_DATE'] < APPLICABLE_DATE)]
    if aggr_df.empty:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    date1 = pd.to_datetime(aggr_df['APPLICABLE_DATE'].values[0])
    date2 = pd.to_datetime(aggr_df['TRAN_DATE'].values[0])
    r = relativedelta.relativedelta(date1, date2)
    months_difference = (r.years * 12) + r.months
    months_in_between = pd.date_range(date2, date1, freq='MS').strftime("%B").tolist()
    tran_months = aggr_df['tran_month'].unique()
    active_months = months_difference - len([x for x in months_in_between if x not in tran_months])

    grouped_df = aggr_df.groupby('tran_month').count().reset_index()
    return months_difference, active_months, np.mean(aggr_df['TRAN_AMOUNT']), np.median(aggr_df['TRAN_AMOUNT']), \
           np.mean(grouped_df['TRAN_AMOUNT'])

# Read the otg data
otg = pd.read_csv('CS_OTG_202305161213.csv', converters={'CIF_ID': str})
otg['TRAN_DATE'] = pd.to_datetime(otg['TRAN_DATE'])
otg['APPLICABLE_DATE'] = pd.to_datetime(otg['APPLICABLE_DATE'])
otg.sort_values(by=['CIF_ID', 'TRAN_DATE'], inplace=True)
otg['tran_month'] = otg['TRAN_DATE'].dt.month_name()

# Aggregate OTG data
otg_unique = otg.drop_duplicates(subset=['CIF_ID']).copy()
otg_unique['first_otg_trans'], otg_unique['number_of_months_active_otg'], otg_unique['avg_otg_tran_amt'], \
    otg_unique['median_otg_tran_amount'], otg_unique['avg_monthly_otg_trans_count'] = \
    zip(*otg_unique.apply(lambda x: otg_aggregates(x.CIF_ID, x.APPLICABLE_DATE, otg), axis=1))

# Save the aggregated data to a CSV file
otg_unique.to_csv('otg_aggregates.csv', index=False)


In [4]:
df_aggregate = pd.read_csv("otg_aggregates.csv",converters={'CIF_ID': str})
df_aggregate.head()

,CIF_ID,TRAN_ID,APPLICABLE_DATE,TRAN_DATE,TRAN_MONTH,TRAN_AMOUNT,tran_month,first_otg_trans,number_of_months_active_otg,avg_otg_tran_amt,median_otg_tran_amount,avg_monthly_otg_trans_count
0,0002636,S450147,2022-05-13,2021-07-24,JUL-21,8000.0,July,9,7,27125.9375,27000.0,1.7777777777777777
1,0003807,S907367,2022-12-23,2022-08-30,AUG-22,10000.0,August,3,3,102155.481481481474475,20000.0,5.40000000000000036
2,0015206,S738171,2022-03-04,2021-12-13,DEC-21,1000.0,December,2,2,260498.31183673470513895,72500.0,12.25
3,0017633,S894991,2022-09-12,2022-02-19,FEB-22,5000.0,February,6,6,6550.15999999999985448,3020.0,9.375
4,0017819,S453764,2022-06-17,2021-09-12,SEP-21,100.0,September,9,9,15028.20952380952439853,5000.0,10.5


In [9]:
cif_id = '0002636'
avg_tran_count_otg = df_aggregate.loc[df_aggregate['CIF_ID'] == cif_id, 'avg_otg_tran_amt'].values

if len(avg_tran_count_otg) > 0:
    print(f"AVG_TRAN_COUNT_OTG for CIF_ID {cif_id}: {avg_tran_count_otg[0]}")
else:
    print(f"CIF_ID {cif_id} not found in the dataframe.")


AVG_TRAN_COUNT_OTG for CIF_ID 0002636: 27125.9375


In [10]:
df_aggregate2 = pd.read_csv("otg_aggregates_new_2.csv",converters={'CIF_ID': str})
df_aggregate2.head()

,Unnamed: 0,CIF_ID,first_otg_trans,number_of_months_active_otg,avg_otg_tran_amt,median_otg_tran_amount,avg_monthly_otg_trans_count
0,53128,0002636,9.0,7.0,27125.9375,27000.0,1.7777777777777777
1,24212,0003807,3.0,3.0,102155.481481481474475,20000.0,5.40000000000000036
2,87595,0015206,2.0,2.0,260498.31183673470513895,72500.0,12.25
3,122652,0017633,6.0,6.0,6550.15999999999985448,3020.0,9.375
4,80088,0017819,9.0,9.0,15028.20952380952439853,5000.0,10.5


In [11]:
cif_id2 = '0002636'
avg_tran_count_otg = df_aggregate2.loc[df_aggregate['CIF_ID'] == cif_id, 'avg_otg_tran_amt'].values

if len(avg_tran_count_otg) > 0:
    print(f"AVG_TRAN_COUNT_OTG for CIF_ID {cif_id}: {avg_tran_count_otg[0]}")
else:
    print(f"CIF_ID {cif_id} not found in the dataframe.")

AVG_TRAN_COUNT_OTG for CIF_ID 0002636: 27125.9375
